Last updated: February 2, 2025

This only requires a CPU and can be used on the free tier of Colab.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/iris-lew/projects/blob/main/project_006_AES/AES_4_custom_loss_function.ipynb)

This notebook is supposed to supplement the other AES notebooks. As I was getting confused over the quadratic weighted kappa description in the [ASAP-AES Kaggle page](https://www.kaggle.com/c/asap-aes). It was used as the evaluation criteria for that competition. I decided to try and implement it myself, and then use the function in the model as a loss function. This would be my first time using custom loss functions, so this is a chronicle of my learning and exploration.

I thought there might be a better way for the model to optimize itself because it would have to minimize based on the loss function. The previous workbooks used categorical cross-entropy loss which measures loss as the difference between the predicted probability distribution and the true probability distribution amongst the labels. Then it used accuracy as its evaluation criteria. Categorical cross-entropy loss can serve as the loss function because there is some distribution of the 0-60 labels in the ASAP-AES dataset, but I was curious as to whether there would be a higher accuracy and if the distributions of the predicted essay scores would change if I used a quadratic weighted kappa loss function. A score of 0 should mean there is only random agreement between the raters (that is the predicted labels and the true labels), but a score of 1 means there is complete agreement.

The quadratic weighted kappa is used as a metric for ordinal classification problems. This means that the variables have a clear order, but there's no guarantee that there is even spacing between the elements (e.g., low, medium, and high). Upon doing further [reading](https://en.wikipedia.org/wiki/Cohen%27s_kappa), I learned that Cohen's kappa is used to measure inter-rater reliablity, but the formula for Cohen's kappa ($\kappa \equiv \frac{p_{o} - p_{e}}{1- p_{e}} = 1 - \frac{1-p_{o}}{1-p{e}}$) is different from the formula for the quadratic weighted kappa found on the Kaggle page ($\kappa = 1 -\frac{\sum_{i,j} w_{i,j}O_{i,j}}{\sum_{i,j}w_{i,j}E_{i,j}}$). [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.cohen_kappa_score.html) has a built-in Cohen's kappa function and is used as a metric in the documentation. [PyTorch ignite](https://pytorch.org/ignite/master/generated/ignite.metrics.CohenKappa.html#ignite.metrics.CohenKappa) also uses the scikit-learn function in the back-end. There are GitHub examples where people have turned the metric into a loss function within PyTorch ([example 1](https://gist.github.com/SupreethRao99/2e85884dad433a6b381f966fea7a6658) and [example 2](https://github.com/Mamiglia/WeightedKappaLoss/blob/main/WeightedKappaLoss.py)), but the examples use the formula for Cohen's kappa instead of the formula in the ASAP-AES Kaggle webpage.

I opted to try and use the GitHub examples as is, but was running into errors in the notebook with the model (AES_5_model_custom_loss.ipynb). This led me to the rewrite the code for the model. I rewrote it to use the formula for quadratic weighted kappa, using [example 1](https://gist.github.com/SupreethRao99/2e85884dad433a6b381f966fea7a6658) as the base. Unfortunately, when I was trying to convert the the $O_{i,j}$ matrix functions into PyTorch functions, this led me to understand that this formula is undifferentiable, but I am able to somehow make the model run even when I just took the regular formula and put it into tensors.

Upon doing further reading, there should be a way to turn the quadratic weighted kappa to a smooth version that is differentiable as shown in this [Medium article](https://medium.com/@wenmin_wu/directly-optimize-quadratic-weighted-kappa-with-tf2-0-482cf0318d74). Since this is an adaptation of someone else's work, I will have to verify that my adaptation is still a quadratic weighted kappa.

My next steps are to convert this into PyTorch and try to use it in my model (AES_5_model_custom_loss.ipynb).

# Setup and Libraries

In [ ]:
### only used for Google Colab
import os
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
%cd /content/gdrive/MyDrive/

Mounted at /content/gdrive
/content/gdrive/MyDrive


I stored the datasets under the data folder and will be operating out of there.

In [ ]:
%cd data

/content/gdrive/MyDrive/data


In [ ]:
# libraries
import numpy as np
import pandas as pd
from functools import reduce
import math

import random
from sklearn.utils import shuffle

import time
import datetime

import transformers
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import TFAutoModel, AutoTokenizer
import torch
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import get_linear_schedule_with_warmup
import torch.nn as nn
from typing import Optional

import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pyplot import figure
import seaborn as sns

In [ ]:
# Global variable settings
seed_val = 0

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

Since this is just for me to figure out the function, I'm only going to load the original ASAP training set and use `rater1_domain1` and `rater2_domain1` to check my work.

In [ ]:
asap_train = pd.read_csv('training_set_rel3.tsv', sep='\t', header=0, encoding="ISO-8859-1")

# As a Metric

I initially decided to write my own version of the Quadratic Weighted Kappa metric to better understand it, and then compare it to the functions found in the [Evaluation Metrics folder](https://github.com/benhamner/ASAP-AES/blob/master/Evaluation_Metrics/Python/score/score.py) that is used for scoring the model submissions. This would verify whether I understood the formula and implemented it correctly.

In [ ]:
def confusion_matrix(rater_a, rater_b,
                     min_rating=None, max_rating=None):
  """
  Returns the confusion matrix between rater's ratings
  """
  assert(len(rater_a)==len(rater_b))
  if min_rating is None:
    min_rating = min(reduce(min, rater_a), reduce(min, rater_b))
  if max_rating is None:
    max_rating = max(reduce(max, rater_a), reduce(max, rater_b))
  num_ratings = max_rating - min_rating + 1
  conf_mat = [[0 for i in range(num_ratings)]
              for j in range(num_ratings)]
  for a,b in zip(rater_a,rater_b):
    conf_mat[a-min_rating][b-min_rating] += 1
  return conf_mat

def histogram(ratings, min_rating=None, max_rating=None):
  """
  Returns the counts of each type of rating that a rater made
  """
  if min_rating is None: min_rating = reduce(min, ratings)
  if max_rating is None: max_rating = reduce(max, ratings)
  num_ratings = max_rating - min_rating + 1
  hist_ratings = [0 for x in range(num_ratings)]
  for r in ratings:
    hist_ratings[r-min_rating] += 1
  return hist_ratings

def quadratic_weighted_kappa(rater_a, rater_b,
                             min_rating = None, max_rating = None):
  """
  Calculates the quadratic weighted kappa score

  QuadraticWeightedKappa calculates the quadratic weighted kappa
  value, which is a measure of inter-rater agreement between two raters
  that provide discrete numeric ratings.  Potential values range from -1
  (representing complete disagreement) to 1 (representing complete
  agreement).  A kappa value of 0 is expected if all agreement is due to
  chance.

  scoreQuadraticWeightedKappa(rater_a, rater_b), where rater_a and rater_b
  each correspond to a list of integer ratings.  These lists must have the
  same length.

  The ratings should be integers, and it is assumed that they contain
  the complete range of possible ratings.

  score_quadratic_weighted_kappa(X, min_rating, max_rating), where min_rating
  is the minimum possible rating, and max_rating is the maximum possible
  rating
  """
  assert(len(rater_a) == len(rater_b))
  if min_rating is None:
    min_rating = min(reduce(min, rater_a), reduce(min, rater_b))
  if max_rating is None:
    max_rating = max(reduce(max, rater_a), reduce(max, rater_b))
  conf_mat = confusion_matrix(rater_a, rater_b,
                              min_rating, max_rating)
  num_ratings = len(conf_mat)
  num_scored_items = float(len(rater_a))

  hist_rater_a = histogram(rater_a, min_rating, max_rating)
  hist_rater_b = histogram(rater_b, min_rating, max_rating)

  numerator = 0.0
  denominator = 0.0

  for i in range(num_ratings):
    for j in range(num_ratings):
      expected_count = (hist_rater_a[i]*hist_rater_b[j] / num_scored_items)
      d = pow(i-j,2.0) / pow(num_ratings-1, 2.0)
      numerator += d*conf_mat[i][j] / num_scored_items
      denominator += d*expected_count / num_scored_items

  return 1.0 - numerator / denominator

def linear_weighted_kappa(rater_a, rater_b,
                          min_rating = None, max_rating = None):
  """
  Calculates the linear weighted kappa

  linear_weighted_kappa calculates the linear weighted kappa
  value, which is a measure of inter-rater agreement between two raters
  that provide discrete numeric ratings.  Potential values range from -1
  (representing complete disagreement) to 1 (representing complete
  agreement).  A kappa value of 0 is expected if all agreement is due to
  chance.

  linear_weighted_kappa(rater_a, rater_b), where rater_a and rater_b
  each correspond to a list of integer ratings.  These lists must have the
  same length.

  The ratings should be integers, and it is assumed that they contain
  the complete range of possible ratings.

  linear_weighted_kappa(X, min_rating, max_rating), where min_rating
  is the minimum possible rating, and max_rating is the maximum possible
  rating
  """
  assert(len(rater_a) == len(rater_b))
  if min_rating is None:
    min_rating = min(reduce(min, rater_a), reduce(min, rater_b))
  if max_rating is None:
    max_rating = max(reduce(max, rater_a), reduce(max, rater_b))
  conf_mat = confusion_matrix(rater_a, rater_b,
                              min_rating, max_rating)
  num_ratings = len(conf_mat)
  num_scored_items = float(len(rater_a))

  hist_rater_a = histogram(rater_a, min_rating, max_rating)
  hist_rater_b = histogram(rater_b, min_rating, max_rating)

  numerator = 0.0
  denominator = 0.0

  for i in range(num_ratings):
    for j in range(num_ratings):
      expected_count = (hist_rater_a[i]*hist_rater_b[j] / num_scored_items)
      d = abs(i-j) / float(num_ratings-1)
      numerator += d*conf_mat[i][j] / num_scored_items
      denominator += d*expected_count / num_scored_items
  return 1.0 - numerator / denominator

def kappa(rater_a, rater_b,
          min_rating = None, max_rating = None):
  """
  Calculates the kappa

  kappa calculates the kappa
  value, which is a measure of inter-rater agreement between two raters
  that provide discrete numeric ratings.  Potential values range from -1
  (representing complete disagreement) to 1 (representing complete
  agreement).  A kappa value of 0 is expected if all agreement is due to
  chance.

  kappa(rater_a, rater_b), where rater_a and rater_b
  each correspond to a list of integer ratings.  These lists must have the
  same length.

  The ratings should be integers, and it is assumed that they contain
  the complete range of possible ratings.

  kappa(X, min_rating, max_rating), where min_rating
  is the minimum possible rating, and max_rating is the maximum possible
  rating
  """
  assert(len(rater_a) == len(rater_b))
  if min_rating is None:
    min_rating = min(reduce(min, rater_a), reduce(min, rater_b))
  if max_rating is None:
    max_rating = max(reduce(max, rater_a), reduce(max, rater_b))
  conf_mat = confusion_matrix(rater_a, rater_b,
                              min_rating, max_rating)
  num_ratings = len(conf_mat)
  num_scored_items = float(len(rater_a))

  hist_rater_a = histogram(rater_a, min_rating, max_rating)
  hist_rater_b = histogram(rater_b, min_rating, max_rating)

  numerator = 0.0
  denominator = 0.0

  for i in range(num_ratings):
    for j in range(num_ratings):
      expected_count = (hist_rater_a[i]*hist_rater_b[j] / num_scored_items)
      if i==j:
        d=0.0
      else:
        d=1.0
      numerator += d*conf_mat[i][j] / num_scored_items
      denominator += d*expected_count / num_scored_items

  return 1.0 - numerator / denominator

def mean_quadratic_weighted_kappa(kappas, weights=None):
  """
  Calculates the mean of the quadratic
  weighted kappas after applying Fisher's r-to-z transform, which is
  approximately a variance-stabilizing transformation.  This
  transformation is undefined if one of the kappas is 1.0, so all kappa
  values are capped in the range (-0.999, 0.999).  The reverse
  transformation is then applied before returning the result.

  mean_quadratic_weighted_kappa(kappas), where kappas is a vector of
  kappa values

  mean_quadratic_weighted_kappa(kappas, weights), where weights is a vector
  of weights that is the same size as kappas.  Weights are applied in the
  z-space
  """
  kappas = np.array(kappas, dtype=float)
  if weights is None:
    weights = np.ones(np.shape(kappas))
  else:
    weights = weights / np.mean(weights)

  # ensure that kappas are in the range [-.999, .999]
  kappas = np.array([min(x, .999) for x in kappas])
  kappas = np.array([max(x, -.999) for x in kappas])

  z = 0.5 * np.log( (1+kappas)/(1-kappas) ) * weights
  z = np.mean(z)
  kappa = (np.exp(2*z)-1) / (np.exp(2*z)+1)
  return kappa

In [ ]:
quadratic_weighted_kappa(asap_train['rater1_domain1'],asap_train['rater2_domain1'])

0.9692509691720163

## Figuring It Out

I constructed the matrices following the steps as outlined in the Kaggle page.

> First, and N-by-N histogram matrix $O$ is constructed over the essay ratings, such that $O_{i,j}$ corresponds to the number of essays that received a rating i by Rater A and a rating j by Rater B.

In [ ]:
N = max(max(asap_train['rater1_domain1']),max(asap_train['rater2_domain1']))

A_B = {}
for i in range(asap_train.shape[0]):
  if str(asap_train['rater1_domain1'][i])+'_'+str(asap_train['rater2_domain1'][i]) not in A_B.keys():
    A_B[str(asap_train['rater1_domain1'][i])+'_'+str(asap_train['rater2_domain1'][i])] = 0
  A_B[str(asap_train['rater1_domain1'][i])+'_'+str(asap_train['rater2_domain1'][i])] += 1
A_B


O = []
for i in range(N+1):
  row = []
  for j in range(N+1):
    if str(i)+'_'+str(j) in A_B.keys():
      row.append(A_B[str(i)+'_'+str(j)])
    else:
      row.append(0)
  O.append(row)

In [ ]:
N

30

In [ ]:
O

[[412,
  42,
  6,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [41,
  1637,
  373,
  11,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [10,
  395,
  1615,
  514,
  20,
  2,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [4,
  12,
  554,
  1789,
  416,
  12,
  2,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [1,
  4,
  12,
  377,
  1609,
  211,
  27,
  5,
  8,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  1,
  0,
  7,
  224,
  359,
  73,
  19,
  18,
  3,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,


> An N-by-N matrix of weights, w, is calculated based on the difference between raters' scores:

> $w_{i,j} = \frac{(i-j)^2}{(N-1)^2}$




In [ ]:
w = []
for i in range(N+1):
  row = []
  for j in range(N+1):
    row.append((pow(i-j,2.0))/(pow(N-1,2.0)))
  w.append(row)
w

[[0.0,
  0.0011890606420927466,
  0.0047562425683709865,
  0.01070154577883472,
  0.019024970273483946,
  0.029726516052318668,
  0.04280618311533888,
  0.05826397146254459,
  0.07609988109393578,
  0.09631391200951249,
  0.11890606420927467,
  0.14387633769322236,
  0.17122473246135553,
  0.20095124851367419,
  0.23305588585017836,
  0.267538644470868,
  0.30439952437574314,
  0.3436385255648038,
  0.38525564803804996,
  0.4292508917954816,
  0.4756242568370987,
  0.5243757431629013,
  0.5755053507728894,
  0.629013079667063,
  0.6848989298454221,
  0.7431629013079667,
  0.8038049940546967,
  0.8668252080856124,
  0.9322235434007135,
  1.0,
  1.070154577883472],
 [0.0011890606420927466,
  0.0,
  0.0011890606420927466,
  0.0047562425683709865,
  0.01070154577883472,
  0.019024970273483946,
  0.029726516052318668,
  0.04280618311533888,
  0.05826397146254459,
  0.07609988109393578,
  0.09631391200951249,
  0.11890606420927467,
  0.14387633769322236,
  0.17122473246135553,
  0.2009512485

> An N-by-N histogram matrix of expected ratings, $E$, is calculated, assuming that there is no correlation between rating scores.  This is calculated as the outer product between each rater's histogram vector of ratings, normalized such that $E$ and $O$ have the same sum.

In [ ]:
d1 = {}
for i in asap_train['rater1_domain1']:
  if i not in d1.keys():
    d1[i] = 0
  d1[i]+=1
v1 = []
for i in range(max(d1.keys())+1):
  if i not in d1.keys():
    v1.append(0)
  else:
    v1.append(d1[i])

d2 = {}
for i in asap_train['rater2_domain1']:
  if i not in d2.keys():
    d2[i] = 0
  d2[i]+=1
v2 = []
for i in range(max(d2.keys())+1):
  if i not in d2.keys():
    v2.append(0)
  else:
    v2.append(d2[i])

In [ ]:
v1[:10]

[462, 2062, 2556, 2789, 2254, 704, 266, 127, 420, 206]

In [ ]:
v2[:10]

[468, 2091, 2561, 2698, 2326, 704, 264, 120, 432, 174]

In [ ]:
len(asap_train['rater1_domain1'])

12976

In [ ]:
462*468

216216

In [ ]:
(462*468)/12976

16.662762022194823

In [ ]:
E = []
for i in range(N+1):
  row = []
  for j in range(N+1):
    row.append(v1[i]*v2[j]/(len(asap_train['rater1_domain1'])))
  E.append(row)
E

[[16.662762022194823,
  74.44836621454994,
  91.182336621455,
  96.06011097410604,
  82.81535141800246,
  25.065351418002464,
  9.399506781750924,
  4.272503082614056,
  15.381011097410605,
  6.195129469790382,
  3.774044389642417,
  3.880856966707768,
  7.797318125770653,
  0.10681257706535142,
  0.2848335388409371,
  3.9520653514180024,
  2.1718557336621456,
  1.9226263871763256,
  3.4180024660912456,
  1.3529593094944512,
  7.726109741060419,
  0.7120838471023427,
  0.7476880394574599,
  0.783292231812577,
  0.17802096177558568,
  1.3885635018495683,
  0.21362515413070285,
  0.07120838471023427,
  0.0,
  0.0,
  0.03560419235511714],
 [74.36929716399507,
  332.278205918619,
  406.9653205918619,
  428.7358199753391,
  369.62176325524047,
  111.87176325524044,
  41.95191122071517,
  19.06905055487053,
  68.64858199753391,
  27.650123304562268,
  16.844327990135636,
  17.321054254007397,
  34.801017262638716,
  0.47672626387176326,
  1.2712700369913688,
  17.638871763255242,
  9.6934340

> From these three matrices, the quadratic weighted kappa is calculated:
$\kappa = 1 -\frac{\sum_{i,j} w_{i,j}O_{i,j}}{\sum_{i,j}w_{i,j}E_{i,j}}$

In [ ]:
numerator = 0
denominator = 0

for i in range(N+1):
  for j in range(N+1):
    numerator += w[i][j]*O[i][j]
    denominator += w[i][j]*E[i][j]
kappa = 1-(numerator/denominator)
kappa

0.9692509691720163

> The Fisher Transformation is approximately a variance-stabilizing transformation and is defined:
$z = \frac{1}{2} \ln \frac{1+\kappa}{1-\kappa}$

In [ ]:
z = .5*np.log((1+kappa)/(1-kappa))
z

2.079775027384743

> Next the mean of the transformed kappa values is calculated in the z-space.  For Essay Set #2, which has scores in two different domains, each transformed kappa is weighted by 0.5.  This means that each dataset has an equally weighted contribution to the final score.  Finally, the reverse transformation is applied to get the average kappa value:
$\kappa = \frac{e^{2z}-1}{e^{2z}+1}$

In [ ]:
average_kappa = (math.exp(2.0*z)-1)/(math.exp(2.0*z)+1)
average_kappa

0.9692509691720163

## Turning it Into a Function

In [ ]:
def quadratic_weighted_kappa_error(y_true, y_pred, nclasses = 60):
  """

  human = human score column. e.g., asap_train['domain1_score']
  machine = predicted score column. e.g., asap_train['domain1_score_pred']
  num_records = number of records in the dataframe. e.g., asap_train.shape[0]

  metric varies from 0 to 1. measurement of agreement between the two raters.
  The quadratic weighted kappa is calculated between the automated scores for the essays
  and the resolved score for human raters on each set of essays.
  The mean of the quadratic weighted kappa is then taken across all sets of essays.
  This mean is calculated after applying the Fisher Transformation to the kappa values.

  """
  N = nclasses

  A_B = {}
  for i in range(len(y_true)):
    if str(y_true[i])+'_'+str(y_pred[i]) not in A_B.keys():
      A_B[str(y_true[i])+'_'+str(y_pred[i])] = 0
    A_B[str(y_true[i])+'_'+str(y_pred[i])] += 1

  O = []
  for i in range(N+1):
    row = []
    for j in range(N+1):
      if str(i)+'_'+str(j) in A_B.keys():
        row.append(A_B[str(i)+'_'+str(j)])
      else:
        row.append(0)
    O.append(row)

  w = []
  for i in range(N+1):
    row = []
    for j in range(N+1):
      row.append((pow(i-j,2.0))/(pow(N-1,2.0)))
    w.append(row)

  d1 = {}
  for i in y_true:
    if i not in d1.keys():
      d1[i] = 0
    d1[i]+=1
  v1 = []

  for i in range(N+1):
    if i not in d1.keys():
      v1.append(0)
    else:
      v1.append(d1[i])
  d2 = {}
  for i in y_pred:
    if i not in d2.keys():
      d2[i] = 0
    d2[i]+=1
  v2 = []
  for i in range(N+1):
    if i not in d2.keys():
      v2.append(0)
    else:
      v2.append(d2[i])
  E = []
  for i in range(N+1):
    row = []
    for j in range(N+1):
      row.append(v1[i]*v2[j]/(len(y_true)))
    E.append(row)

  print(w)
  print(O)
  numerator = 0.0
  denominator = 0.0
  for i in range(N+1):
    for j in range(N+1):
      numerator += w[i][j]*O[i][j]
      denominator += w[i][j]*E[i][j]
  kappa = 1-(numerator/denominator)

  return kappa

In [ ]:
quadratic_weighted_kappa_error(asap_train['rater1_domain1'],asap_train['rater2_domain1'])

[[0.0, 0.0002872737719046251, 0.0011490950876185005, 0.002585463947141626, 0.004596380350474002, 0.007181844297615628, 0.010341855788566504, 0.01407641482332663, 0.018385521401896008, 0.023269175524274634, 0.028727377190462512, 0.03476012640045964, 0.04136742315426602, 0.04854926745188164, 0.05630565929330652, 0.06463659867854064, 0.07354208560758403, 0.08302212008043666, 0.09307670209709853, 0.10370583165756966, 0.11490950876185005, 0.12668773340993966, 0.13904050560183856, 0.15196782533754669, 0.16546969261706407, 0.17954610744039068, 0.19419706980752657, 0.2094225797184717, 0.22522263717322608, 0.2415972421717897, 0.2585463947141626, 0.2760700948003447, 0.2941683424303361, 0.31284113760413673, 0.33208848032174665, 0.35191037058316577, 0.37230680838839414, 0.39327779373743177, 0.41482332663027865, 0.4369434070669348, 0.4596380350474002, 0.4829072105716748, 0.5067509336397586, 0.5311692042516518, 0.5561620224073542, 0.5817293881068658, 0.6078713013501867, 0.6345877621373168, 0.6618787

0.9692509691720163

In [ ]:
1-quadratic_weighted_kappa_error(asap_train['rater1_domain1'],asap_train['rater2_domain1'])

[[0.0, 0.0002872737719046251, 0.0011490950876185005, 0.002585463947141626, 0.004596380350474002, 0.007181844297615628, 0.010341855788566504, 0.01407641482332663, 0.018385521401896008, 0.023269175524274634, 0.028727377190462512, 0.03476012640045964, 0.04136742315426602, 0.04854926745188164, 0.05630565929330652, 0.06463659867854064, 0.07354208560758403, 0.08302212008043666, 0.09307670209709853, 0.10370583165756966, 0.11490950876185005, 0.12668773340993966, 0.13904050560183856, 0.15196782533754669, 0.16546969261706407, 0.17954610744039068, 0.19419706980752657, 0.2094225797184717, 0.22522263717322608, 0.2415972421717897, 0.2585463947141626, 0.2760700948003447, 0.2941683424303361, 0.31284113760413673, 0.33208848032174665, 0.35191037058316577, 0.37230680838839414, 0.39327779373743177, 0.41482332663027865, 0.4369434070669348, 0.4596380350474002, 0.4829072105716748, 0.5067509336397586, 0.5311692042516518, 0.5561620224073542, 0.5817293881068658, 0.6078713013501867, 0.6345877621373168, 0.6618787

0.030749030827983748

Even though my code is messy and isn't the most efficient way, programming it myself allows me to learn what the function is doing.

# As a Loss Function

The difference between a loss function and a metric is that a loss function needs to be differentiable so that back-propagation can occur. PyTorch stores the data in tensors which may have gradients enabled. If gradients are enabled, the functions keep track of the operations performed and uses the gradients for back-propagation.

Thus, I need update the functions so that they use Torch functions and are compatible with Torch tensors.

I updated the `rater1_domain1` and `rater2_domain1` columns so they are tensors as that is what the model will be working with.

In [ ]:
tensor1 = torch.Tensor(asap_train['rater1_domain1'].tolist())
tensor1.requires_grad_()
tensor2 = torch.Tensor(asap_train['rater2_domain1'].tolist())
tensor2.requires_grad_()

tensor([ 4.,  4.,  3.,  ..., 26., 20., 20.], requires_grad=True)

In [ ]:
max(asap_train['rater1_domain1']) # trying out some functions

30

In [ ]:
torch.max(tensor1).item() # trying out some functions

30.0

In [ ]:
t1list = tensor1.tolist() # trying out some functions
t1list[:4]

[4.0, 5.0, 4.0, 5.0]

In [ ]:
list(map(int, t1list))[:4] # trying out some functions

[4, 5, 4, 5]

In [ ]:
print(tensor1)
print(tensor2)

tensor([ 4.,  5.,  4.,  ..., 20., 20., 20.], requires_grad=True)
tensor([ 4.,  4.,  3.,  ..., 26., 20., 20.], requires_grad=True)


The following blocks is before I discovered the `torch.histc` function.

In [ ]:
empty = []
m = torch.max(tensor1).item()
for i in range(int(m+1)):
    empty.append((tensor1 == i).unsqueeze(0))
v1_tensor = torch.cat(empty,dim=0).sum(dim=1)
v1_tensor = v1_tensor.double()
v1_tensor.requires_grad_()

tensor([4.6200e+02, 2.0620e+03, 2.5560e+03, 2.7890e+03, 2.2540e+03, 7.0400e+02,
        2.6600e+02, 1.2700e+02, 4.2000e+02, 2.0600e+02, 1.4800e+02, 1.0200e+02,
        1.8200e+02, 1.3000e+01, 9.0000e+00, 1.1300e+02, 4.6000e+01, 7.4000e+01,
        7.3000e+01, 3.0000e+01, 2.4500e+02, 2.2000e+01, 1.4000e+01, 1.9000e+01,
        6.0000e+00, 2.7000e+01, 1.0000e+00, 2.0000e+00, 1.0000e+00, 0.0000e+00,
        3.0000e+00], dtype=torch.float64, requires_grad=True)

In [ ]:
empty = []
m = torch.max(tensor2).item()
for i in range(int(m+1)):
    empty.append((tensor2 == i).unsqueeze(0))
v2_tensor = torch.cat(empty,dim=0).sum(dim=1)
v2_tensor = v2_tensor.float()
v2_tensor.requires_grad_()

tensor([4.6800e+02, 2.0910e+03, 2.5610e+03, 2.6980e+03, 2.3260e+03, 7.0400e+02,
        2.6400e+02, 1.2000e+02, 4.3200e+02, 1.7400e+02, 1.0600e+02, 1.0900e+02,
        2.1900e+02, 3.0000e+00, 8.0000e+00, 1.1100e+02, 6.1000e+01, 5.4000e+01,
        9.6000e+01, 3.8000e+01, 2.1700e+02, 2.0000e+01, 2.1000e+01, 2.2000e+01,
        5.0000e+00, 3.9000e+01, 6.0000e+00, 2.0000e+00, 0.0000e+00, 0.0000e+00,
        1.0000e+00], requires_grad=True)

The $O$ matrix cannot be derived from only using Torch functions, so I decided to put them in a loop.

In [ ]:
tensor1 = tensor1.int()
tensor2 = tensor2.int()
conf_mat = [[0 for i in range(61)]
              for j in range(61)]
for a,b in zip(tensor1,tensor2):
  conf_mat[a][b] += 1
tensor = torch.zeros(60,60)
tensor = tensor.new_tensor(conf_mat, requires_grad=True)
tensor

tensor([[ 412.,   42.,    6.,  ...,    0.,    0.,    0.],
        [  41., 1637.,  373.,  ...,    0.,    0.,    0.],
        [  10.,  395., 1615.,  ...,    0.,    0.,    0.],
        ...,
        [   0.,    0.,    0.,  ...,    0.,    0.,    0.],
        [   0.,    0.,    0.,  ...,    0.,    0.,    0.],
        [   0.,    0.,    0.,  ...,    0.,    0.,    0.]], requires_grad=True)

I adapted the following code block from this [GitHub example](https://gist.github.com/SupreethRao99/2e85884dad433a6b381f966fea7a6658). I left the comments in to mark my thoughts and my uncertainties, as well as to show what I changed.

In [ ]:
class WeightedKappaLoss(nn.Module):
  """
  Implements Weighted Kappa Loss. Weighted Kappa Loss was introduced in the
  [Weighted kappa loss function for multi-class classification
    of ordinal data in deep learning]
    (https://www.sciencedirect.com/science/article/abs/pii/S0167865517301666).

  Weighted Kappa is widely used in Ordinal Classification Problems. The loss
  value lies in $[-\infty, \log 2]$, where $\log 2$ means the random prediction

  Usage: loss_fn = WeightedKappaLoss(num_classes = NUM_CLASSES)

  """
  def __init__(
      self,
      num_classes: int,
      mode: Optional[str] = 'quadratic',
      name: Optional[str] = 'cohen_kappa_loss',
      epsilon: Optional[float]= 1e-10):

    """Creates a `WeightedKappaLoss` instance.
        Args:
          num_classes: Number of unique classes in your dataset.
          weightage: (Optional) Weighting to be considered for calculating
            kappa statistics. A valid value is one of
            ['linear', 'quadratic']. Defaults to 'quadratic'.
          name: (Optional) String name of the metric instance.
          epsilon: (Optional) increment to avoid log zero,
            so the loss will be $ \log(1 - k + \epsilon) $, where $ k $ lies
            in $ [-1, 1] $. Defaults to 1e-10.
        Raises:
          ValueError: If the value passed for `weightage` is invalid
            i.e. not any one of ['linear', 'quadratic']
        """

    super(WeightedKappaLoss, self).__init__()
    self.num_classes = num_classes
    if mode == 'quadratic':
      self.y_pow = 2
    if mode == 'linear':
      self.y_pow = 1

    self.epsilon = epsilon

  def kappa_loss(self, y_pred, y_true):
    num_classes = self.num_classes
    y = torch.eye(num_classes).to(device)
    y_true_orig = y_true
    y_true = y[y_true]

    y_true = y_true.float()
    y_true_orig = y_true_orig.float()


    # testing
    y_true_orig_int = y_true_orig.int()
    y_pred_int = y_pred.int()
    conf_mat = [[0 for i in range(num_classes)]
              for j in range(num_classes)]
    for a,b in zip(y_true_orig_int,y_pred_int):
      conf_mat[a][b] += 1
    tensor = torch.zeros(num_classes,num_classes)
    O = tensor.new_tensor(conf_mat, device=device)
    # end testing

    repeat_op = torch.Tensor(list(range(num_classes))).unsqueeze(1).repeat((1, num_classes)).to(device)
    repeat_op_sq = torch.square((repeat_op - repeat_op.T))
    weights = repeat_op_sq / ((num_classes - 1) ** 2)

    # y_pred.requires_grad_() ## added this in to hopefully get the backwards function to work
    y_pred_hist = torch.histc(y_pred, bins=num_classes, min=0, max=num_classes) # i added this in
    # pred_ = y_pred_hist ** self.y_pow # changed from y_pred to y_pred_hist
    pred_ = y_pred_hist ##### no need to square it according to the formula.
    pred_norm = pred_ / (self.epsilon + torch.reshape(torch.sum(pred_, 0), [-1, 1])) # used to be torch.sum(pred_,1)

    hist_rater_a = torch.sum(pred_norm, 0) # histogram of normalized predictions
    y_true_hist = torch.histc(y_true_orig, bins=num_classes, min=0, max=num_classes) # i added this in. this is histogram of true values.

    # fix conf_mat so it is a matrix
    a = torch.Tensor(list(pred_)).unsqueeze(1).repeat((1, num_classes)).to(device)
    # pred_.expand(num_classes) # replace above line with this? try it?
    conf_mat = torch.mul(a,y_true_hist) # used to be torch.matmul(pred_norm.T, y_true). now (pred_norm, y_true_hist)
    conf_mat = conf_mat/len(y_true_orig)

    bsize = y_pred.size(0)

    nom = torch.sum(weights * O) ##### need to work in 61 classes
    # expected_probs = torch.matmul(torch.reshape(hist_rater_a, [num_classes, 1]),
    #                               torch.reshape(y_true_hist, [1, num_classes]))
    # denom = torch.sum(weights * expected_probs / bsize)
    denom = torch.sum(weights * conf_mat) # originally the numerator
    return nom / (denom + self.epsilon)

  def forward(self, y_pred, y_true):
    return self.kappa_loss(y_pred,y_true)

loss_fn = WeightedKappaLoss(
    num_classes = 61,
    mode = 'quadratic'
)

In [ ]:
device="cpu"
loss = loss_fn(tensor1.float(), tensor2.int())
loss

tensor(0.0307)

This is the same value as the metric version, so I am reasonably assured that the formulas aren't completely wrong.

I then wanted to try it on a batch of of data to see if it was calculating properly on new data.

In [ ]:
tensor1 = torch.FloatTensor([55., 11., 21., 31., 31., 55., 55., 11., 31., 11., 55., 21., 55., 55.,
                             11., 55., 55., 19., 27., 55., 22., 55., 55., 55., 11., 55., 19., 31.,
                             55., 55., 55., 55.]) # predicted
tensor2 = torch.IntTensor([16,  1,  2,  4,  0,  8,  4,  3,  1,  9, 11,  7,  0,  2, 37,  2,  9,  3,
                           18,  3,  4,  3, 14,  0,  2,  3,  0, 35, 10,  3, 40, 24]) # truth

In [ ]:
loss_fn(tensor1, tensor2).item()

0.9910561442375183

# Smooth Version

I found a [medium article](https://medium.com/@wenmin_wu/directly-optimize-quadratic-weighted-kappa-with-tf2-0-482cf0318d74) that contains a smooth version of the quadratic weighted kappa so I would be able to use it as a loss function. It is written in TensorFlow so I needed to convert it to PyTorch.

There were a few things that I changed in order to use it:
1.   The a & b tensors were updated from the shape of y_true to the number of classes. This is because I am expecting to be working on 61x61 matrices.
2.   As I am working on 61x61 matrices, I updated the `y_true` and `y_pred` matrices so that I would be working with the counts of `y_true` and `y_pred`.

    a.   For example, `labels` is created with a matrix multiplication of `y_true` and `col_label_vec`. `col_label_vec` is initially a matrix of shape (`num_classes`,1). This would be (61,1) in my case. The length `y_true` would vary based on batch size and the length of my training data, hence it made more sense to use the counts of `y_true`.

***I am in the process of verifying that I am still using a quadratic weighted kappa and that this is a valid loss function.***



In [ ]:
class WeightedKappaLoss(nn.Module):
  """
  """
  def __init__(
      self,
      num_classes: int,
      epsilon: Optional[float]= 1e-10):

    """
    """
    super(WeightedKappaLoss, self).__init__()
    self.num_classes = num_classes
    self.epsilon = epsilon

  def kappa_loss(self, y_pred, y_true):
    num_classes = self.num_classes

    label_vec = torch.arange(start=0, end=num_classes).type(torch.float32).to(device)
    row_label_vec = torch.reshape(label_vec, [1, num_classes]).to(device)
    col_label_vec = torch.reshape(label_vec, [num_classes, 1]).to(device)

    y_true = y_true.float().to(device)
    y_true_hist = torch.histc(y_true, bins=num_classes, min=0, max=num_classes) # i added this in. this is histogram of true values.
    labels = torch.matmul(y_true_hist, col_label_vec) # this becomes one number: tensor([278.])


    a = torch.tile(labels, [1, num_classes]) # updated it to be num_classes instead of the shape of y_true
    b = torch.tile(row_label_vec, [num_classes,1]) # updated it to be num_classes instead of the shape of y_true
    c = torch.tile(col_label_vec, [1, num_classes])


    weight = torch.pow(a - b,2)
    weight = torch.div(weight, (num_classes-1)**2) # (61,61) matrix

    weight_mat = torch.div(torch.pow(c-b,2),((num_classes-1)**2)) # (61,61) matrix

    y_pred_hist = torch.histc(y_pred, bins=num_classes, min=0, max=num_classes) # i added this in. this is histogram of predicted values.
    d = weight*y_pred_hist # (61,61) matrix

    numerator = torch.sum(d) # originally tf.reduce_sum

    f = torch.matmul(weight_mat, y_pred_hist) # originally tf.reduce_sum # shape is 61
    denominator = torch.matmul(y_true_hist,f)
    denominator = torch.div(denominator, num_classes)

    return torch.log(numerator / denominator + self.epsilon)

  def forward(self, y_pred, y_true):
    return self.kappa_loss(y_pred,y_true)

loss_fn = WeightedKappaLoss(
    num_classes = 61
)

loss_fn(tensor1, tensor2)

tensor(8.5068)